In [5]:
from utils.load_data_for_eda import load_data
import numpy as np

df = load_data('data/commodity_prices.csv')
df['log_Modal_Price'] = df['Modal_Price'].apply(lambda x: np.log(x) if x > 0 else np.nan)

In [7]:
"""
Compute within-product effect sizes (eta squared η² and omega squared ω²) for
Season, Market, and Year on price.

Why:
- You want to know how strongly each factor (Season, Market, Year) explains price
  variation *within each product*, without forcing a global mixed model.
- η² is the share of total variance explained by a factor in a one-way ANOVA.
- ω² is a bias-corrected version of η² (more conservative, especially for small n).

Inputs expected in your DataFrame `df`:
- Columns: 'Product_Type', 'Season', 'Market', 'Year', and a price column.
- Recommended price column: 'log_Modal_Price'. If unavailable, you can use
  'Modal_Price' (non-logged), but results may be more skewed.

Outputs:
- A DataFrame with one row per product and columns:
    ['n_obs', 'n_seasons', 'n_markets', 'n_years',
     'eta2_season', 'omega2_season', 'eta2_market', 'omega2_market',
     'eta2_year', 'omega2_year',
     'usable_season', 'usable_market', 'usable_year']

Notes:
- These are *one-factor* effect sizes per product. They are comparable across
  products, but they don't sum to 1 within a product (factors can overlap).
- 'usable_*' is True if the factor has at least 2 levels and each level has
  at least 2 observations (a minimal reliability check).
- Missing data are dropped pairwise for each factor.
"""
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Tuple, Dict

import numpy as np
import pandas as pd

try:
    from scipy.stats import f as f_dist
except Exception:  # scipy may not be installed in every environment
    f_dist = None


@dataclass
class OneWayEffectSizes:
    n: int
    k: int
    ss_between: float
    ss_within: float
    ss_total: float
    ms_between: float
    ms_within: float
    df_between: int
    df_within: int
    F: float
    p_value: Optional[float]
    eta2: float
    omega2: float


def _oneway_effect_sizes(y: pd.Series, x: pd.Series) -> Optional[OneWayEffectSizes]:
    """Compute one-way ANOVA effect sizes (η², ω²) for y grouped by x.

    Returns None if not enough data (k<2 or any group size <2 or degenerate variances).
    """
    # Drop NA pairwise
    df = pd.DataFrame({"y": y, "x": x}).dropna()
    if df.shape[0] < 3:
        return None

    # Group stats
    grouped = df.groupby("x")
    k = grouped.ngroups
    if k < 2:
        return None

    sizes = grouped.size().astype(int)
    if (sizes < 2).any():  # minimal reliability: each level has >=2 obs
        return None

    means = grouped["y"].mean()
    vars_ = grouped["y"].var(ddof=1)

    # Basic sums of squares
    N = int(sizes.sum())
    grand_mean = df["y"].mean()
    ss_between = float(np.sum(sizes.values * (means.values - grand_mean) ** 2))
    # Within = sum of (n_g - 1) * s_g^2
    ss_within = float(np.sum((sizes.values - 1) * vars_.values))
    ss_total = ss_between + ss_within

    df_between = k - 1
    df_within = N - k
    if df_within <= 0:
        return None

    ms_between = ss_between / df_between if df_between > 0 else np.nan
    ms_within = ss_within / df_within if df_within > 0 else np.nan

    # Guard against zero within variance
    if not np.isfinite(ms_within) or ms_within <= 0:
        return None

    F = ms_between / ms_within if np.isfinite(ms_between) else np.nan
    if f_dist is not None and np.isfinite(F):
        p_value = float(f_dist.sf(F, df_between, df_within))
    else:
        p_value = None

    # Effect sizes
    eta2 = ss_between / ss_total if ss_total > 0 else np.nan
    # Omega squared (bias-corrected). May be slightly negative; clip at 0.
    omega2_num = ss_between - df_between * ms_within
    omega2_den = ss_total + ms_within
    omega2 = omega2_num / omega2_den if omega2_den > 0 else np.nan
    omega2 = float(max(0.0, omega2)) if np.isfinite(omega2) else np.nan

    return OneWayEffectSizes(
        n=N,
        k=k,
        ss_between=ss_between,
        ss_within=ss_within,
        ss_total=ss_total,
        ms_between=ms_between,
        ms_within=ms_within,
        df_between=df_between,
        df_within=df_within,
        F=float(F),
        p_value=p_value,
        eta2=float(eta2),
        omega2=float(omega2),
    )


def compute_dependency_profiles(
    df: pd.DataFrame,
    price_col: str = "log_Modal_Price",
    product_col: str = "Product_Type",
    factors: Tuple[str, ...] = ("Season", "Market", "Year"),
    min_obs_per_product: int = 10,
) -> pd.DataFrame:
    """Compute η² and ω² per factor within each product.

    Parameters
    ----------
    df : DataFrame with product, factors, and price column.
    price_col : Name of price column (prefer log_Modal_Price).
    product_col : Name of product identifier column.
    factors : Iterable of factor column names to evaluate.
    min_obs_per_product : Skip products with fewer observations than this (set to
        0 to disable). This is a *soft* filter; rows are kept but flagged.

    Returns
    -------
    profiles : DataFrame indexed by product with columns:
        n_obs, n_<factor>, eta2_<factor>, omega2_<factor>, usable_<factor>
    """
    # Ensure needed columns exist
    needed = {product_col, price_col, *factors}
    missing = needed.difference(df.columns)
    if missing:
        raise KeyError(f"Missing required columns: {sorted(missing)}")

    records: Dict[str, Dict[str, float]] = {}

    for prod, g in df.dropna(subset=[product_col, price_col]).groupby(product_col):
        rec: Dict[str, float] = {}
        rec["n_obs"] = int(g.shape[0])

        # Per-factor counts and effect sizes
        for fac in factors:
            # Unique level count within product
            rec[f"n_{fac.lower()}s"] = int(g[fac].dropna().nunique())

            effects = _oneway_effect_sizes(g[price_col], g[fac])
            if effects is None:
                rec[f"eta2_{fac.lower()}"] = np.nan
                rec[f"omega2_{fac.lower()}"] = np.nan
                rec[f"F_{fac.lower()}"] = np.nan
                rec[f"p_{fac.lower()}"] = np.nan
                rec[f"usable_{fac.lower()}"] = False
            else:
                rec[f"eta2_{fac.lower()}"] = effects.eta2
                rec[f"omega2_{fac.lower()}"] = effects.omega2
                rec[f"F_{fac.lower()}"] = effects.F
                rec[f"p_{fac.lower()}"] = effects.p_value
                rec[f"usable_{fac.lower()}"] = True

        # Soft reliability flag for the whole product
        rec["passes_min_obs"] = bool(rec["n_obs"] >= min_obs_per_product)

        records[prod] = rec

    profiles = (
        pd.DataFrame.from_dict(records, orient="index")
        .rename_axis(product_col)
        .reset_index()
    )

    # Column order (friendly)
    order = [
        product_col,
        "n_obs",
        "n_seasons",
        "n_markets",
        "n_years",
        "eta2_season",
        "omega2_season",
        "eta2_market",
        "omega2_market",
        "eta2_year",
        "omega2_year",
        "usable_season",
        "usable_market",
        "usable_year",
        "F_season",
        "p_season",
        "F_market",
        "p_market",
        "F_year",
        "p_year",
        "passes_min_obs",
    ]
    # Keep only those that exist
    order = [c for c in order if c in profiles.columns]
    profiles = profiles[order]

    return profiles


# ------------------------------
# Example usage (commented):
# ------------------------------
profiles = compute_dependency_profiles(df, price_col="log_Modal_Price")
#
# # Inspect strongest season-driven products
profiles.sort_values("eta2_season", ascending=False).head(20)
#
# # Save to CSV
# # profiles.to_csv("product_dependency_profiles.csv", index=False)
# 


,Product_Type,n_obs,n_seasons,n_markets,n_years,eta2_season,omega2_season,eta2_market,omega2_market,eta2_year,...,usable_season,usable_market,usable_year,F_season,p_season,F_market,p_market,F_year,p_year,passes_min_obs
50,Coconut|Coconut|FAQ,73,3,1,2,0.764034,0.754748,NaN,NaN,0.763152,...,True,False,True,113.326534,1.121300e-22,NaN,NaN,228.770229,6.688490e-24,True
15,Banana|Other|Medium,83,3,3,1,0.650779,0.639258,0.346013,0.326990,NaN,...,True,True,False,74.540592,5.296911e-19,21.163278,4.195362e-08,NaN,NaN,True
69,Garlic|Garlic|FAQ,69,3,1,2,0.642738,0.628510,NaN,NaN,0.092891,...,True,False,True,59.369304,1.772339e-15,NaN,NaN,6.861049,1.088581e-02,True
119,Tomato|Deshi|Non-FAQ,122,3,1,1,0.604214,0.595581,NaN,NaN,NaN,...,True,False,False,90.833579,1.119251e-24,NaN,NaN,NaN,NaN,True
96,Papaya|Other|Large,646,4,2,3,0.544504,0.541991,0.177046,0.175544,0.413011,...,True,True,True,255.817692,3.534049e-109,138.546873,4.179533e-29,226.210706,4.115889e-75,True
94,Orange|Other|Large,149,3,1,1,0.534722,0.526670,NaN,NaN,NaN,...,True,False,False,83.895401,5.533797e-25,NaN,NaN,NaN,NaN,True
93,Onion|Small|FAQ,188,4,4,2,0.523313,0.514209,0.161022,0.146674,0.360562,...,True,True,True,67.332563,1.972659e-29,11.771506,4.328034e-07,104.880604,8.381757e-20,True
86,Mango|Other|Medium,209,4,1,2,0.498231,0.489689,NaN,NaN,0.172687,...,True,False,True,67.851487,1.628787e-30,NaN,NaN,43.207568,3.928171e-10,True
105,Potato|Big|FAQ,76,3,1,2,0.484614,0.467196,NaN,NaN,0.054513,...,True,False,True,34.320725,3.110543e-11,NaN,NaN,4.266551,4.237194e-02,True
48,Coconut Seed|Other|FAQ,292,4,1,3,0.461409,0.454948,NaN,NaN,0.757013,...,True,False,True,82.242918,1.852824e-38,NaN,NaN,450.182108,1.647176e-89,True
